# TM10007 Assignment template -- ECG data

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [ ]:
# Run this to use from Colab environment
!git clone https://github.com/jveenland/tm10007_ml.git

import time
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from tqdm import tqdm  # Progress bar
from scipy.stats import uniform, loguniform

# Extract dataset
with zipfile.ZipFile('/content/tm10007_ml/ecg/ecg_data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/tm10007_ml/ecg')


# Load dataset
data = pd.read_csv('ecg_data.csv', index_col=0)

print(f'The number of samples: {len(data.index)}')
print(f'The number of features: {len(data.columns) - 1}')  # Excluding label column

# Extract features and labels
X = data.iloc[:, :-1].values  # All columns except the last one
y = data.iloc[:, -1].values   # Last column as labels

# Scale the features
scaler = RobustScaler(quantile_range=(30, 70))
X_scaled = scaler.fit_transform(X)

# Apply PCA to reduce dimensionality while preserving 99% of variance
pca = PCA(n_components=0.99)
X_pca = pca.fit_transform(X_scaled)

print(f'Reduced number of features after PCA: {X_pca.shape[1]}')

# Split into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Define hyperparameter search space for RBF kernel
param_distributions = {
    'C': loguniform(0.01, 500),  # Log-uniform distribution for better coverage of large ranges
    'gamma': loguniform(0.0001, 10),  # Log-uniform for better granularity
    'class_weight': ['balanced', None],  # Adjusts for class imbalance
    'shrinking': [True, False],  # Use shrinking heuristic or not
    'tol': uniform(1e-5, 1e-2),  # Tolerance for stopping
}


# Number of iterations for RandomizedSearchCV
n_iter_search = 500  # Increased to explore more combinations

# Initialize SVM model with RBF kernel
svm = SVC(kernel='rbf')

# Perform Randomized Search
random_search = RandomizedSearchCV(
    estimator=svm,
    param_distributions=param_distributions,
    n_iter=n_iter_search,  # Large number of trials
    scoring='accuracy',
    cv=5,  # 5-fold cross-validation
    n_jobs=-1,
    random_state=42
)

# Timing the process
start_time = time.time()

with tqdm(total=n_iter_search, desc="Running Hyperparameter Optimization") as pbar:
    random_search.fit(X_train, y_train)
    pbar.close()

elapsed_time = time.time() - start_time  # Calculate elapsed time

# Get best parameters and best score
best_params = random_search.best_params_
best_model = random_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test)

# Print results
print("\nBest hyperparameters found:", best_params)
print(f"Best cross-validation accuracy: {random_search.best_score_:.4f}")
print(f"Test set accuracy: {test_accuracy:.4f}")
print(f"Time taken: {elapsed_time:.2f} seconds")


fatal: destination path 'tm10007_ml' already exists and is not an empty directory.
The number of samples: 827
The number of features: 9000
